In [1]:
import torch
import numpy as np

In [2]:
device=('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
torch.cuda.is_available() #gpu 가 없거나 있어도 setting X

False

In [4]:
data_set=np.loadtxt('data/ThoraricSurgery3.csv', delimiter=',')
data_set=torch.from_numpy(data_set)

In [5]:
data_set.shape #torch. -> tensor type

torch.Size([470, 17])

In [6]:
X=data_set[:, :-1]
y=data_set[:, -1]

In [7]:
X.shape

torch.Size([470, 16])

In [8]:
y.shape

torch.Size([470])

In [9]:
import torch.nn as nn

class Surgery(nn.Module):
    def __init__(self): #constructor
        super(Surgery, self).__init__()
        self.hidden_linear=nn.Linear(16, 30) #keras의 dense
        self.hidden_activation=nn.ReLU()
        self.output_linear=nn.Linear(30,1)
        self.output_activation=nn.Sigmoid()
        
    def forward(self, x):
        x=self.hidden_linear(x)
        x=self.hidden_activation(x)
        x=self.output_linear(x)
        x=self.output_activation(x)
        
        return x
    
    def predict(self, x):
        pred=self.forward(x)
        if pred >= 0.5:
            return 1
        else: 
            return 0

In [10]:
model=Surgery().to(device) #bias=False (bias 안씀)
model

Surgery(
  (hidden_linear): Linear(in_features=16, out_features=30, bias=True)
  (hidden_activation): ReLU()
  (output_linear): Linear(in_features=30, out_features=1, bias=True)
  (output_activation): Sigmoid()
)

In [11]:
param_gen=model.named_parameters()
param_gen

<generator object Module.named_parameters at 0x00000140434CABA0>

In [12]:
#next(param_gen) #hidden layer의 가중치 파라미터 16*30 #requires_grad=True: 학습해야할 param

In [13]:
def count_parameters(model):
    total_param = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_param = np.prod(param.size())
            if param.dim() > 1:
                print(name, ':', ' x '.join(str(x) for x in list(param.size())[::-1]), '=', num_param)
            else:
                print(name, ':', num_param)
                print('-' * 40)
            total_param += num_param
    print('total:', total_param)

In [14]:
count_parameters(model)

hidden_linear.weight : 16 x 30 = 480
hidden_linear.bias : 30
----------------------------------------
output_linear.weight : 30 x 1 = 30
output_linear.bias : 1
----------------------------------------
total: 541


In [17]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [38]:
ds=TensorDataset(X, y)
dataloader= DataLoader(ds, batch_size=10)
optimizer=optim.Adam(model.parameters())

n_epochs=10

for epoch in range(n_epochs):
    for data, label in dataloader:
        out=model(data.type(torch.FloatTensor).to(device)) #forward propagation
        loss=nn.BCELoss()(out, label.type(torch.FloatTensor).unsqueeze(1).to(device)) #unsqueeze(=expanddim 차원 늘리기) ; 현재는 0,1번째 차원 사이 차원 증가 #calculate loss (배치 내 손실 총합)
        #torch.nn.BCELoss는 PyTorch 라이브러리에서 제공하는 클래스로, 이진 분류(Binary Classification) 문제에서 사용되는 이진 교차 엔트로피(Binary Cross Entropy) 손실 함수를 구현한 것입니다. 
        # BCELoss는 예측값과 타겟값 사이의 교차 엔트로피를 계산하여 손실 값을 얻는데 사용됩니다.
        loss.backward() #개별 파라미터 기울기 계산
        optimizer.step() #개별 파라미터 갱신
        optimizer.zero_grad() #optimizer가 계산해서 저장해 놓은 기울기값 clear #"Pytorch에서는 gradients값들을 추후에 backward를 해줄때 계속 더해주기 때문"에 우리는 항상 backpropagation을 하기전에 gradients를 zero로 만들어주고 시작을 해야합니다.
    print("Epoch: %d, Loss: %.4f" %(epoch, float(loss)))

Epoch: 0, Loss: 0.4169
Epoch: 1, Loss: 0.4039
Epoch: 2, Loss: 0.3959
Epoch: 3, Loss: 0.3904
Epoch: 4, Loss: 0.3868
Epoch: 5, Loss: 0.3845
Epoch: 6, Loss: 0.3820
Epoch: 7, Loss: 0.3795
Epoch: 8, Loss: 0.3779
Epoch: 9, Loss: 0.3759


In [27]:
for i in dataloader:
    print(i)

[tensor([[ 1.0000,  2.8800,  2.1600,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,
          1.0000,  3.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000, 60.0000],
        [ 2.0000,  3.4000,  1.8800,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000, 51.0000],
        [ 2.0000,  2.7600,  2.0800,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000, 59.0000],
        [ 2.0000,  3.6800,  3.0400,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 54.0000],
        [ 2.0000,  2.4400,  0.9600,  2.0000,  0.0000,  1.0000,  0.0000,  1.0000,
          1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000, 73.0000],
        [ 2.0000,  2.4800,  1.8800,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 51.0000],
        [ 2.0000,  4.